In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### 목표

이 튜토리얼에서는 다음 방법을 학습합니다.

- Google 검색 결과를 그라운딩으로 LLM 텍스트 및 채팅 모델 응답 생성
- 그라운딩이 설정되지 않은 LLM 응답 결과와 그라운딩 설정된 LLM 응답 비교
- Vertex AI Search에서 데이터 저장소를 생성하고 사용하여 사용자 지정 문서 및 데이터에서 응답 그라운딩 생성
- Vertex AI Search 결과를 그라운딩으로 LLM 텍스트 및 채팅 모델 응답 생성

이 튜토리얼에서는 다음과 같은 Google Cloud AI 서비스 및 리소스를 사용합니다.

- Vertex AI
- Vertex AI Search

수행 단계는 다음과 같습니다.

- 다양한 예제에 대한 LLM 및 프롬프트 구성
- Vertex AI의 생성 텍스트 및 채팅 모델에 예제 프롬프트 전송
- 자체 데이터로 Vertex AI Search에 데이터 저장소 설정
- 다양한 수준의 그라운딩 설정(그라운딩 설정 없음, 웹 그라운딩 설정, 데이터 저장소 그라운딩 설정)을 사용하여 예제 프롬프트 전송

## 시작하기 전에

### Google Cloud 프로젝트 설정

**다음 단계는 노트북 환경에 관계없이 필수입니다.**

1. [Google Cloud 프로젝트를 선택하거나 생성합니다](https://console.cloud.google.com/cloud-resource-manager). 계정을 처음 생성하면 컴퓨팅/스토리지 비용에 사용할 수 있는 $300의 무료 크레딧이 제공됩니다.
1. [프로젝트에 대한 결제가 활성화되어 있는지 확인합니다](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Vertex AI 및 Vertex AI Search API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,discoveryengine.googleapis.com)를 활성화합니다.
1. 이 노트북을 로컬에서 실행하는 경우 [Cloud SDK](https://cloud.google.com/sdk)를 설치해야 합니다.

### Python용 Google Gen AI SDK 설치

이 노트북을 실행하는 데 필요한 다음 패키지를 설치하세요.

In [ ]:
%pip install --upgrade --quiet google-genai
%pip install --upgrade --quiet langchain-google-vertexai

### Google Cloud 계정 인증

이 노트북을 실행하는 경우 환경을 인증해야 합니다. 이를 위해 아래 새 셀을 실행하세요. 링크를 클릭하여 구글 클라우드 인증을 합니다. 로그인 후 나오는 key를 아래 쉘에 붙여넣습니다.


In [ ]:
!gcloud auth application-default login --no-launch-browser


### Google Cloud 프로젝트 정보 설정 및 클라이언트 생성

Vertex AI를 사용하려면 기존 Google Cloud 프로젝트가 있어야 하며 [Vertex AI API를 활성화](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)해야 합니다.

[프로젝트 및 개발 환경 설정](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)에 대해 자세히 알아보세요.

**프로젝트 ID를 모르는 경우** 다음을 시도해 보세요.
* `gcloud config list`를 실행합니다.
* `gcloud projects list`를 실행합니다.
* 지원 페이지: [프로젝트 ID 찾기](https://support.google.com/googleapi/answer/7014113)를 참조하세요.

Vertex AI에서 사용하는 `LOCATION` 변수를 변경할 수도 있습니다. [Vertex AI 리전](https://cloud.google.com/vertex-ai/docs/general/locations)에 대해 자세히 알아보세요.

In [ ]:
import os

PROJECT_ID = ""  # @param {type: "string"}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION =  "us-central1" # @param {type: "string"}

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### 라이브러리 가져오기

In [ ]:
from IPython.display import Markdown, display
from google.genai.types import (
    ApiKeyConfig,
    AuthConfig,
    EnterpriseWebSearch,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleMaps,
    GoogleSearch,
    LatLng,
    Part,
    Retrieval,
    RetrievalConfig,
    Tool,
    ToolConfig,
    VertexAISearch,
)

### Helper functions

In [ ]:
import re
from IPython.display import display, Markdown
import traceback

def display_grounding_data(response):
           """
           (Production Version) A robust function to parse and display grounding data
           from the native Vertex AI SDK, handling various edge cases.
           """
           try:
               # --- 1. 응답 객체에서 핵심 데이터 추출 ---
               candidate = response.candidates[0]
               text_content = candidate.content.parts[0].text
               metadata = candidate.grounding_metadata

               chunks = getattr(metadata, 'grounding_chunks', [])
               supports = getattr(metadata, 'grounding_supports', [])
               queries = getattr(metadata, 'web_search_queries', getattr(metadata, 'retrieval_queries', None)) # Changed to None default

               # --- 2. 텍스트에 인용(Citation) 삽입 ---
               final_text_parts = []
               citation_map = {}  # 인용 번호와 chunk 인덱스 매핑

               if not supports:
                   final_text_parts.append(text_content)
               else:
                   text_bytes = text_content.encode('utf-8')
                   last_index = 0
                   # end_index 순서대로 정렬하여 순서가 섞여도 정상 처리
                   sorted_supports = sorted(supports, key=lambda s: getattr(s.segment, 'end_index', 0))

                   for support in sorted_supports:
                       segment = getattr(support, 'segment', None)
                       if not segment: continue

                       end_index = int(getattr(segment, 'end_index', 0))
                       indices = getattr(support, 'grounding_chunk_indices', [])

                       final_text_parts.append(text_bytes[last_index:end_index].decode('utf-8'))
                       footnote = "".join([f"[{i+1}]" for i in indices])
                       final_text_parts.append(f"**{footnote}**")

                       # 인용 매핑 저장
                       for idx in indices:
                           citation_map[idx + 1] = idx

                       last_index = end_index

                   final_text_parts.append(text_bytes[last_index:].decode('utf-8'))

               final_text = "".join(final_text_parts)

               # --- 3. 근거 소스(Source) 섹션 생성 ---
               source_parts = []
               if chunks:
                   source_parts.append("\n\n" + "="*50)
                   source_parts.append("\n## 📚 Grounding Sources (검색 근거 자료)\n")

                   # 각 chunk를 인용 번호와 함께 표시
                   for i, chunk in enumerate(chunks, 1):
                       source_parts.append(f"\n### [{i}] 출처 {i}")

                       # Web context 처리
                       web_context = getattr(chunk, 'web', None)
                       if web_context:
                           domain = getattr(web_context, 'domain', 'Unknown')
                           title = getattr(web_context, 'title', 'Untitled')
                           uri = getattr(web_context, 'uri', '')

                           source_parts.append(f"\n- **도메인**: {domain}")
                           source_parts.append(f"\n- **제목**: {title}")
                           if uri:
                               # Vertex AI Search redirect URL 처리
                               if 'vertexaisearch.cloud.google.com' in uri:
                                   source_parts.append(f"\n- **링크**: [검색 결과 보기]({uri})")
                               else:
                                   source_parts.append(f"\n- **링크**: [{domain}]({uri})")

                       # Retrieved context 처리 (파일 그라운딩 검색인 경우)
                       retrieved_context = getattr(chunk, 'retrieved_context', None)
                       if retrieved_context:
                           uri = getattr(retrieved_context, 'uri', '')
                           title = getattr(retrieved_context, 'title', 'Untitled')

                           source_parts.append(f"\n- **문서**: {title}")
                           if uri:
                               if uri.startswith("gs://"):
                                   link = uri.replace("gs://", "https://storage.googleapis.com/", 1)
                                   source_parts.append(f"\n- **저장소**: [GCS 링크]({link})")
                               else:
                                   source_parts.append(f"\n- **경로**: {uri}")

                       source_parts.append("\n")

               # --- 4. 검색 쿼리 표시 ---
               if queries: # Added check if queries is not None
                   source_parts.append("\n" + "-"*40)
                   source_parts.append("\n### 🔍 Web Search Queries (웹 검색 쿼리)\n")
                   for q in queries:
                       source_parts.append(f"\n- `{q}`")

               # --- 5. 최종 결과 출력 ---
               print("\n" + "="*50)
               print("📝 **생성된 답변 (Citations 포함)**")
               print("="*50)
               display(Markdown(final_text + "".join(source_parts)))

               # --- 6. 디버그 정보 (선택적) ---
               print("\n" + "="*50)
               print("📊 **Grounding 통계**")
               print(f"- 검색된 소스 수: {len(chunks)}개")
               print(f"- 인용된 구간 수: {len(supports)}개")
               if queries: # Added check if queries is not None
                   print(f"- 사용된 검색 쿼리: {len(queries)}개")
               print("="*50)

           except Exception as e:
               # 예상치 못한 에러 발생 시 상세 내용 출력
               print("\n--- ❌ AN UNEXPECTED ERROR OCCURRED ---")
               print(f"Error: {e}")
               traceback.print_exc()

Vertex AI에서 Gemini 모델을 초기화합니다.

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## 예시: Google 검색 결과를 활용한 그라운딩

이 예시에서는 그라운딩이 없는 LLM 응답과 Google 검색 결과를 활용한 응답을 비교해 보겠습니다. 가장 최근의 일식에 대한 질문을 해 보겠습니다.

In [ ]:
PROMPT = "한국에서 다음 일식은 언제인가요?"

### 그라운딩 없이 텍스트 생성

그라운딩 없이 LLM에 예측 요청:

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
)

display(Markdown(response.text))

### Google 검색 결과를 그라운딩 한 텍스트 생성

`Tool`에 `GoogleSearch`를 포함하는 `tools` 키워드 인수를 추가하여 Gemini가 먼저 프롬프트를 사용하여 Google 검색을 수행한 다음, 웹 검색 결과를 그라운딩하여 답변을 구성하도록 할 수 있습니다.

응답의 각 `Candidate`에 대해 검색어와 [검색 진입점](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points)을 사용할 수 있습니다.

In [ ]:
google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[google_search_tool]),
)

display_grounding_data(response)

그라운딩이 없는 응답은 LLM에서 일식에 대한 제한된 정보만 제공합니다. 반면, 웹 검색 결과에서 접지가 적용된 응답은 LLM에서 접지 요청과 함께 반환되는 웹 검색 결과에서 가장 최신 정보를 포함합니다.

### Google 검색 결과를 그라운딩으로 다중 모달 입력을 사용한 텍스트 생성

Gemini는 다중 모달 입력을 사용하여 그라운딩 응답도 생성할 수 있습니다. 이 에펠탑 이미지로 시도해 보겠습니다.

![파리](https://storage.googleapis.com/github-repo/generative-ai/gemini/grounding/paris.jpg)

In [ ]:
PROMPT = "이 위치의 현재 기온은 얼마입니까?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://github-repo/generative-ai/gemini/grounding/paris.jpg",
            mime_type="image/jpeg",
        ),
        PROMPT,
    ],
    config=GenerateContentConfig(
        tools=[google_search_tool],
    ),
)

display_grounding_data(response)

## 예시: Enterprise Web Search를 이용한 Grounding

Google 검색을 이용한 Grounding은 Google 검색을 사용하여 웹 검색을 수행합니다. 이 서비스의 일환으로 Google 검색은 고객 쿼리를 로깅할 수 있습니다([Google Cloud 서비스별 약관 19.k항 참조](https://cloud.google.com/terms/service-terms)). 이는 금융이나 의료와 같이 규제가 엄격한 산업의 고객의 규정 준수 요건을 충족하지 못하는 경우가 많습니다.

Enterprise Web Search는 이러한 요건을 충족합니다. 고객이 Enterprise Web Search를 사용하여 웹에서 Grounding을 수행할 때, 고객 데이터를 로깅하지 않고 리전 내 VPC SC 및 ML 처리를 완벽하게 지원합니다. Enterprise Web Search Grounding은 미국 및 EU 다중 리전에서 사용할 수 있습니다.

Enterprise Web Search Grounding의 요청 및 응답 형식은 Google 검색을 이용한 Grounding과 매우 유사합니다.

### Gemini 모델 호환성

Enterprise Web Search는 Grounding을 지원하는 모든 Gemini 2.5 모델과 호환됩니다. Gemini 2.5 Flash는 다중 모드 입력(예: 이미지, 문서, 비디오)을 지원합니다.

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}
PROMPT = "리오넬 메시와 크리스티아누 호날두의 경력을 비교해 보세요"

enterprise_web_search_tool = Tool(enterprise_web_search=EnterpriseWebSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[enterprise_web_search_tool]),
)

display_grounding_data(response)

Gemini 2.5 Pro를 사용하면 그라운딩은 여러 순차 검색도 수행할 수 있습니다.

In [ ]:
MODEL_ID = "gemini-2.5-pro"  # @param {type: "string"}
PROMPT = "2024년 올림픽 100m 스프린트 우승자는 어디에서 태어났습니까?"

enterprise_web_search_tool = Tool(enterprise_web_search=EnterpriseWebSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[enterprise_web_search_tool]),
)

display_grounding_data(response)

## 예시: 사용자 지정 문서 및 데이터 그라운딩 접근

이 예시에서는 그라운딩 접근이 없는 LLM 응답과 [Vertex AI Search의 검색 앱 결과](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest)를 그라운딩으로 하는 응답을 비교합니다.

데이터 저장소에는 가상 은행인 Cymbal Bank의 내부 문서가 포함됩니다. 이러한 문서는 공개 인터넷에서 사용할 수 없으므로 Gemini 모델은 기본적으로 해당 문서에 대한 정보를 포함하지 않습니다.

### Vertex AI Search에서 데이터 저장소 만들기

이 예시에서는 은행의 내부 문서 몇 개가 포함된 Google Cloud Storage 버킷을 사용합니다. 출장 예약 관련 문서, 이번 회계연도 전략 계획, 그리고 회사에서 제공하는 다양한 직무를 설명하는 HR 문서가 있습니다.

Vertex AI Search 문서의 튜토리얼 단계에 따라 다음을 수행합니다.

1. GCS 폴더 `gs://cloud-samples-data/gen-app-builder/search/cymbal-bank-employee`에서 문서를 로드하는 [비정형 데이터로 데이터 저장소를 만듭니다](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#unstructured-data).
2. 해당 데이터 저장소에 연결된 [검색 앱을 만듭니다](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app). 데이터 저장소 내에서 인덱싱된 레코드를 검색할 수 있도록 **Enterprise Edition 기능**도 활성화해야 합니다.

**참고:** 데이터 저장소는 Gemini에서 사용하는 것과 동일한 프로젝트에 있어야 합니다.

이 노트북을 따라 코드를 작성할 수도 있습니다. [Vertex AI Search 데이터 저장소 및 앱 만들기](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb)

데이터 저장소를 생성했으면 앱 ID를 얻어 아래에 입력하세요.

참고: 데이터 저장소를 그라운드와 함께 사용하려면 데이터 수집이 완료될 때까지 기다려야 합니다. 자세한 내용은 [데이터 저장소 만들기](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es)를 참조하세요.

In [ ]:
VERTEX_AI_SEARCH_PROJECT_ID = ""  # @param {type: "string"}
VERTEX_AI_SEARCH_REGION = "global"  # @param {type: "string"}
# Replace this with your App (Engine) ID from Vertex AI Search
VERTEX_AI_SEARCH_APP_ID = ""  # @param {type: "string"}

VERTEX_AI_SEARCH_ENGINE_NAME = f"projects/{VERTEX_AI_SEARCH_PROJECT_ID}/locations/{VERTEX_AI_SEARCH_REGION}/collections/default_collection/engines/{VERTEX_AI_SEARCH_APP_ID}"

이제 회사 문화에 대한 질문을 할 수 있습니다.

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}
PROMPT = "회사 문화는 어떤가요?"

### 그라운드 없이 텍스트 생성

그라운드 없이 LLM에 답변 요청:

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
)

display(Markdown(response.text))

### Vertex AI 검색 결과에 그라운딩한 텍스트 생성

이제 `tools` 키워드 인수에 `grounding.VertexAISearch()`라는 그라운딩 도구를 추가하여 LLM이 먼저 검색 앱 내에서 검색을 수행한 다음 관련 문서를 그라운딩으로 답변을 구성하도록 지시할 수 있습니다.

In [ ]:
vertex_ai_search_tool = Tool(
    retrieval=Retrieval(
        vertex_ai_search=VertexAISearch(engine=VERTEX_AI_SEARCH_ENGINE_NAME)
    )
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[vertex_ai_search_tool]),
)

display_grounding_data(response)

그라운딩 없는 답변에는 문의하신 회사에 대한 맥락이 전혀 없습니다. 반면, Vertex AI 검색 결과를 그라운딩으로 한 답변에는 제공된 문서의 정보와 해당 정보의 인용이 포함되어 있습니다.

<div class="alert alert-block alert-warning">
<b>⚠️ 중요 참고 사항:</b><br>
<br>
<b>이전 셀을 실행할 때 오류가 발생하는 경우:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;이 샘플 노트북이 Vertex AI Search의 데이터 저장소에서 작동하려면<br>
&nbsp;&nbsp;&nbsp;&nbsp;<a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_data_store">데이터 저장소</a> <b>및</b> Vertex AI에 연결된 <a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app">검색 앱</a>을 만들어야 합니다. 검색.<br>
&nbsp;&nbsp;&nbsp;&nbsp;데이터 저장소만 생성하는 경우, 이전 요청은 데이터 저장소에 대한 쿼리를 실행할 때 오류를 반환합니다.
<br><br>
<b>이전 셀을 실행할 때 빈 응답이 반환되는 경우:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;접지 기능이 있는 데이터 저장소를 사용하려면 데이터 수집이 완료될 때까지 기다려야 합니다.<br>
&nbsp;&nbsp;&nbsp;&nbsp;자세한 내용은 <a href="https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es">데이터 저장소 생성</a>을 참조하세요.
</div>
</div>

## 예시: 그라운딩 채팅 응답

Vertex AI에서 채팅 대화를 사용할 때 그라운딩을 사용할 수도 있습니다. 이 예시에서는 그라운딩이 없는 LLM 응답과 Google 검색 결과 및 Vertex AI 검색의 데이터 저장소를 그라운딩 하는 응답을 비교합니다.

In [ ]:
PROMPT = "Vertex AI에서 관리되는 데이터 세트란 무엇인가요?"
PROMPT_FOLLOWUP = "어떤 유형의 데이터를 사용할 수 있나요?"

### Google 검색 결과를 그라운딩으로 한 채팅 세션

이제 `tools` 키워드 인수에 `GoogleSearch`라는 Tool 속성을 추가하여 채팅 모델이 먼저 프롬프트를 사용하여 Google 검색을 수행한 다음, 웹 검색 결과를 그라운딩으로 답변을 구성하도록 지시할 수 있습니다.

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(tools=[Tool(google_search=GoogleSearch())]),
)

display(Markdown("## Prompt"))
display(Markdown(f"> {PROMPT}"))
response = chat.send_message(PROMPT)
display_grounding_data(response)

display(Markdown("---\n"))

display(Markdown("## Follow-up Prompt"))
display(Markdown(f"> {PROMPT_FOLLOWUP}"))
response = chat.send_message(PROMPT_FOLLOWUP)
display_grounding_data(response)

### Vertex AI 검색 결과에 그라운딩한 채팅 세션

이제 `tools` 키워드 인수에 `VertexAISearch`라는 그라운딩 도구를 추가하여 채팅 세션이 먼저 사용자 지정 검색 앱 내에서 검색을 수행한 후 관련 문서를 그라운딩으로 답변을 구성하도록 지시할 수 있습니다.

In [ ]:
PROMPT = "출장은 어떻게 예약하나요?"
PROMPT_FOLLOWUP = "더 자세히 알려주세요."

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        tools=[
            Tool(
                retrieval=Retrieval(
                    vertex_ai_search=VertexAISearch(engine=VERTEX_AI_SEARCH_ENGINE_NAME)
                )
            )
        ]
    ),
)

display(Markdown("## Prompt"))
display(Markdown(f"> {PROMPT}"))
response = chat.send_message(PROMPT)
display_grounding_data(response)

display(Markdown("---\n"))

display(Markdown("## Follow-up Prompt"))
display(Markdown(f"> {PROMPT_FOLLOWUP}"))
response = chat.send_message(PROMPT_FOLLOWUP)
display_grounding_data(response)

## 정리

이 노트북에서 사용된 리소스에 대해 Google Cloud 계정에 요금이 청구되지 않도록 하려면 다음 단계를 따르세요.

1. 불필요한 Google Cloud 요금이 청구되지 않도록 [Google Cloud 콘솔](https://console.cloud.google.com/)을 사용하여 필요하지 않은 프로젝트를 삭제하세요. [프로젝트 관리 및 삭제](https://cloud.google.com/resource-manager/docs/creating-managing-projects)에 대한 Google Cloud 문서를 참조하세요.
1. 기존 Google Cloud 프로젝트를 사용한 경우, 계정에 요금이 청구되지 않도록 생성한 리소스를 삭제하세요. 자세한 내용은 [Vertex AI Search의 데이터 저장소에서 데이터 삭제](https://cloud.google.com/generative-ai-app-builder/docs/delete-datastores) 문서를 참조한 후 데이터 저장소를 삭제하세요.
2. Google Cloud Console에서 [Vertex AI Search API](https://console.cloud.google.com/apis/api/discoveryengine.googleapis.com) 및 [Vertex AI API](https://console.cloud.google.com/apis/api/aiplatform.googleapis.com)를 비활성화합니다.